# Objective
The objective for this assignment is to take the in-class Multi-Layer Perceptron code we used for classification and modify it for regression.

# Approach
## MLP Class
We write an MLP class that inherits `torch.nn.Module`, the basic Neural Network module containing the required functions we'll use for Linear Regression.

The `torch.nn.Sequential` class creates a sequential container that allows us to manually call a sequence of modules. In effect, it enables us to transform the container as needed, like creating three `torch.nn.Linear` layers. The input to the first layer should be the number of features and the output of the last layer should be 1. In this case, we'll call the Sigmoid activation function to see a non-linear fit to the data. This will be graphed later in the report.

In [ ]:
import torch

class MLP(torch.nn.Module):
	def __init__(self, num_features):
		super().__init__()
		self.all_layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_features, 5),
            torch.nn.Sigmoid(),
            # 2nd hidden layer
            torch.nn.Linear(5, 2),
            torch.nn.Sigmoid(),						  
            # output layer
            torch.nn.Linear(2, 1),
        )

	def forward(self, x):
		logits = self.all_layers(x)
		return logits

## Dataset, Data normalization, and Dataloader

Given the data below

In [ ]:
X_train = torch.tensor([245.0, 273.0, 304.0, 331.0, 347.0, 360.0, 387.0, 438.0, 493.0, 547.0]).view(-1,1)
y_train = torch.tensor([232.3, 241.1, 257.4, 301.5, 324.6, 350.2, 362.3, 389.0, 398.2, 401.8])

We can create a `MyDataset` class modeled after the one we discussed in the lecture. This time, instead of inheriting from the `Dataset` class, we'll inherit from the `TensorDataset` class as our data is already tensorized.

The `MyDataset` class is a map-style dataset and needs to implement the `__getItem__()` and `__len__()` protocols. By defining these methods, we enable the use of the `DataLoader` utility class, allowing us to easily iterate through the dataset during our training loop.

In [ ]:
from torch.utils.data import TensorDataset 

class MyDataset(TensorDataset):
	def __init__(self, X, y):
		self.features = X
		self.labels = y

	def __getitem__(self, index):
		x = self.features[index]
		y = self.labels[index]
		return x, y
	
	def __len__(self):
		return self.labels.shape[0]

The classification example during lecture did not use normalized data because it was already centered at zero, and had relatively small values.

In our case, we will normalize the data by doing z-score standardization using the mean and standard deviation of the given data. We'll also later use these values to make predictions when plotting the regression curve.

We then put our normalized data into the `MyDataset` class, which is then inserted into the `DataLoader` utility class for training.

In [ ]:
from torch.utils.data import DataLoader 

X_mean, X_std = X_train.mean(), X_train.std()
y_mean, y_std = y_train.mean(), y_train.std()

X_normalized = (X_train - X_mean) / X_std
y_normalized = (y_train - y_mean) / y_std

train_ds = MyDataset(X_normalized, y_normalized)

train_loader = DataLoader(
	dataset=train_ds,
	batch_size=5,
	shuffle=True,
)